In [ ]:
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.6/33.6 MB 37.8 MB/s eta 0:00:00


In [ ]:
import cv2
import mediapipe as mp
import os
from google.colab.patches import cv2_imshow

In [ ]:
# Открываем видеофайл
video_path = '/content/Andriy_gedan_barai_right_side_view.mp4'
video = cv2.VideoCapture(video_path)

In [ ]:
# Создаем объект MediaPipe Pose
pose = mp.solutions.pose.Pose()
frame_count = 0

In [ ]:
# Создаем директории для сохранения кадров и координат
save_frames_dir = '/content/saved_frames'
os.makedirs(save_frames_dir, exist_ok=True)

save_coordinates_dir = '/content/saved_coordinates'
os.makedirs(save_coordinates_dir, exist_ok=True)

In [ ]:
'''#Очистка директорий
!rm -r /content/saved_frames/*
!rm -r /content/saved_coordinates/*'''

In [ ]:
# Обработка кадров и получение результатов позы
while video.isOpened():
    ret, frame = video.read()
    if not ret:
        break

    # Преобразуем кадр в RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Обрабатываем кадр и получаем результаты позы
    results = pose.process(frame_rgb)

    # Сохраняем кадр
    frame_path = os.path.join(save_frames_dir, f'frame_{frame_count}.jpg')
    cv2.imwrite(frame_path, frame)

    # Сохраняем координаты точек левой руки
    if results.pose_landmarks is not None:
        # Получаем координаты точек левой руки, включая плечо
        left_hand_landmarks = [results.pose_landmarks.landmark[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER],
                               results.pose_landmarks.landmark[mp.solutions.pose.PoseLandmark.LEFT_ELBOW],
                               results.pose_landmarks.landmark[mp.solutions.pose.PoseLandmark.LEFT_WRIST],
                               results.pose_landmarks.landmark[mp.solutions.pose.PoseLandmark.LEFT_THUMB],
                               results.pose_landmarks.landmark[mp.solutions.pose.PoseLandmark.LEFT_INDEX],
                               results.pose_landmarks.landmark[mp.solutions.pose.PoseLandmark.LEFT_PINKY]]

        # Выводим название кадра
        cv2.putText(frame, f'Frame: {frame_count}',
                 (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        # Создаем файл для сохранения координат
        filename = f"coordinates_{frame_count}.txt"
        save_path = os.path.join(save_coordinates_dir, filename)

        # Сохраняем координаты точек левой руки в файл
        with open(save_path, 'w') as f:
            f.write(f'Frame: {frame_count}\n')
            for i, landmark in enumerate(left_hand_landmarks):
                x = landmark.x
                y = landmark.y
                z = landmark.z
                f.write(f'Point {i+1}: X: {x:.2f} Y: {y:.2f} Z: {z:.2f}\n')

        # Выводим координаты точек левой руки в консоль
        print(f'Frame: {frame_count}')
        for i, landmark in enumerate(left_hand_landmarks):
            x = landmark.x
            y = landmark.y
            z = landmark.z
            print(f'Point {i+1}: X: {x:.2f} Y: {y:.2f} Z: {z:.2f}')

        mp.solutions.drawing_utils.draw_landmarks(
            frame, results.pose_landmarks, mp.solutions.pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp.solutions.drawing_utils.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
            connection_drawing_spec=mp.solutions.drawing_utils.DrawingSpec(color=(0, 255, 0), thickness=2)
)

        for i, landmark in enumerate(left_hand_landmarks):
            x = int(landmark.x * frame.shape[1])
            y = int(landmark.y * frame.shape[0])
            if i == 0:
        # Плечо
                cv2.circle(frame, (x, y), 5, (0, 0, 255), -1)  # Красный цвет
            elif i == 1 or i == 2:
        # Локоть и кисть
                cv2.circle(frame, (x, y), 5, (0, 0, 255), -1)  # Красный цвет
            else:
                cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)  # Зеленый цвет

            if i > 0:
        # Соединительные линии на левой руке
                cv2.line(frame, (int(left_hand_landmarks[i-1].x * frame.shape[1]), int(left_hand_landmarks[i-1].y * frame.shape[0])),
                 (x, y), (0, 0, 255), 2)  # Красный цвет

    # Отображаем кадр
    cv2_imshow(frame)

    frame_count += 1

video.release()

In [ ]:
from natsort import natsorted

In [ ]:
# Путь к папке с сохраненными кадрами
frames_folder = '/content/saved_frames'


# Получение списка файлов в папке с кадрами
frame_files = sorted(os.listdir(frames_folder))

# Чтение первого кадра для определения размеров видео
first_frame = cv2.imread(os.path.join(frames_folder, frame_files[0]))
height, width, _ = first_frame.shape

# Создание объекта VideoWriter для сохранения видео
output_file = 'saved_video/output.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter(output_file, fourcc, 30, (width, height))

# Чтение каждого кадра из папки и запись его в видео
for frame_file in frame_files:
    frame_path = os.path.join(frames_folder, frame_file)
    frame = cv2.imread(frame_path)
    video_writer.write(frame)









In [ ]:
# Закрытие объекта VideoWriter
video_writer.release()

In [ ]:
# Воспроизведение видео на экране
video_capture = cv2.VideoCapture(output_file)
while True:
    ret, frame = video_capture.read()
    if not ret:
        break
    cv2.imshow('Video', frame)
    if cv2.waitKey(1) == ord('q'):
        break

In [ ]:
# Закрытие окон и освобождение ресурсов
cv2.destroyAllWindows()
video_capture.release()